In [ ]:
import pandas as pd
import pickle

import sys
sys.path.append("../")
import src.soporte as sp

In [ ]:
# Importamos nuestros datos
df = pd.read_csv("../data/train.csv")
df.head(2)

In [ ]:
# Eliminamos la columna que no vamos a usar
df.drop(["id"], axis = 1, inplace = True)

In [ ]:
# Eliminamos las variables poco relevantes, en mi caso, carat engloba "x", "y", "z", voy a probar con este enfoque
df.drop(["depth", "table", "x", "y", "z"], axis = 1, inplace = True)

In [ ]:
# Con una función, eliminamos los outliers de manera personalizada
especial = {"x" : {"top" : 15},
            "y" : {"top" : 20},
            "z" : {"top" : 10}}
out = sp.detectar_outliers(df, "price", especial)

In [ ]:
# Sustituimos los outliers por nulos para después procesarlos
df_sin_out = sp.tratar_outliers(df, out, "null")

In [ ]:
# Tratamos los nulos con el método IterativeImputer
df2 = sp.tratamiento_nulos_num(df_sin_out, metodo = "iterative", respuesta = "price")

In [ ]:
# Realizamos encoding de las variables categóricas de forma label a ver que pasa
encoding = {"cut": "label",
        "color": "label",
        "clarity": "label"}
df_enconded = sp.encoder(df2, encoding, modelo = 3)

In [ ]:
# Buscamos los parámetros base posibles para nuestros árboles
parametros = sp.mejores_parametros_num(df_enconded, "price")

In [ ]:
parametros

In [ ]:
new_parametros = {"max_depth" : [2, 3, 4 ,5 , 6]+ [x for x in range(6, parametros["max_depth"] + 1, 4)],
                  "max_features": [1, 2, 3, 4],
                  "min_samples_leaf": [20, 40, 60, 80],
                  "min_samples_split": [20, 40, 60, 80]}

In [ ]:
# Entrenamos cuatro modelos distintos para comparar sus métricas
metricas = sp.modelos_num(df_enconded, "price", lista = ["knn", "gradient"], parametros_tree = new_parametros, scoring = "neg_mean_squared_error", modelo = 2)

In [ ]:
# En este caso, cargo las métricas para no regenerar los modelos(mucho tiempo)
with open('../data/metricas_2.pkl', 'rb') as metri:
    metricas = pickle.load(metri)
# Comparamos las métricas, en este caso, los modelos son interesantes.    
metricas

In [ ]:
# Entrenamos de nuevo esos modelos pero esta vez con todos los datos
metricas = sp.modelos_num(df_enconded, "price", lista = ["knn", "gradient"], parametros_tree = new_parametros, scoring = "neg_mean_squared_error", comparativa = False, modelo = 2)